#######################################################

شغال ع المعالج

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes sentence-transformers PyPDF2 langchain chromadb langchain_community


In [9]:
import PyPDF2
from langchain_core.documents import Document
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    documents = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            chunks = split_text_into_chunks(text, max_chars=512)

            for chunk in chunks:
                metadata = {
                    "source": pdf_path,
                    "page": page_num + 1
                }
                documents.append(Document(page_content=chunk, metadata=metadata))

    return documents

def split_text_into_chunks(text, max_chars=512):
    """Split text into parts of a specified size"""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1
        if current_length + word_length > max_chars:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Process PDF files
documents = []
docs_dir = "./documents"
for filename in os.listdir(docs_dir):
    if filename.endswith('.pdf'):
        filepath = os.path.join(docs_dir, filename)
        documents.extend(extract_text_from_pdf(filepath))

# Initialize embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

# Device map configuration
device_map = "auto"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configure pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
    torch_dtype=torch.bfloat16,
    device_map=device_map
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-13-d0e467664e1b>:28: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [14]:
prompt_template = """
<s>[INST] You are an assistant for answering questions using provided context.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.

Question: {question}
Context: {context} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

# Test the system
question = "Are aphids a pest?"
print(qa_chain.invoke(question)['result'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: Aphids are considered pests because they can cause significant damage to plants by sucking sap from leaves and stems, which can weaken the plant and reduce its ability to grow and produce fruit. However, as mentioned in the context, having beneficial insects like lady beetles that prey on aphids can help manage their populations without relying on pesticides.


################################################################